# Delta Lake Basics Lab

Delta Lake allows you to read, write and query data in data lakes in an efficient manner.

## In this lesson you:
* Create a new Delta Lake from aggregate data of an existing Delta Lake
* UPSERT records into a Delta lake
* Append new data to an existing Delta Lake

## Audience
* Primary Audience: Data Engineers
* Secondary Audience: Data Analysts and Data Scientists

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater

## Datasets Used
We will use online retail datasets from `/mnt/training/online_retail`

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtraineastus.blob.core.windows.net/

Created user-specific database

Using the database aleksei_cherniaev_if_lv_db .

All done!

-sandbox
<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> **This lab depends upon the complete execution of the notebook titled "Open-Source-Delta-Lake" which registered the `customer_data_delta` table. If this table doesn't exist, run the cell below.**

In [0]:
%run "./Includes/Delta-Lab-1-Prep"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtraineastus.blob.core.windows.net/

Created user-specific database

Using the database aleksei_cherniaev_if_lv_db .

All done!

Out[9]: False

Out[11]: DataFrame[]

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
14,4,0,10


Because we'll be calculating some aggregates in this notebook, we'll change our partitions after shuffle from the default `200` to `8` (which is a good number for the 8 node cluster we're currently working on).

In [0]:
%python

sqlContext.setConf("spark.sql.shuffle.partitions", "8")

-sandbox
## What is a table? 
Before we continue, we need to address a semantic concern addressed by the [Databricks docs](https://docs.databricks.com/user-guide/tables.html#view-databases-and-tables):

> A Databricks table is a collection of structured data. Tables are equivalent to Apache Spark DataFrames.

Generally, the distinction between tables and DataFrames in Spark can be summarized by discussing scope and persistence:
- Tables are defined at the **workspace** level and **persist** between notebooks.
- DataFrames are defined at the **notebook** level and are **ephemeral**.

When we discuss **Delta tables**, we are always talking about collections of structured data that persist between notebooks. Importantly, we do not need to register a directory of files to Spark SQL in order to refer to them as a table. The directory of files itself _is_ the table; registering it with a useful name to Spark SQL just gives us easy accessing to querying these underlying data.

A **Delta Lake** can be thought of as a collection of one or many Delta tables. Generally, an entire elastic storage container will be dedicated to a single Delta Lake, and data will be enriched and cleaned as it is promoted through pre-defined logic.

<img alt="Best Practice" title="Best Practice" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-blue-ribbon.svg"/> To make Delta tables easily accessible, register them using Spark SQL. Use table ACLs to control access in workspaces shared by many diverse parties within an organization.

### Creating a new Delta table

You business intelligence team wants to create a dashboard to track the total number of orders made by customers globally. Many of your customers are international retailers, and have the same customer ID.

Because you batch process your data each day, you've decided to create a workflow that will update their numbers when you run your reports each night.

In this notebook, we'll start by transforming existing data stored in Delta to create a new Delta table for your BI team's dashboard. Then, we'll create processes to append new data to our full records as well as updating the Delta table for the BI team.

### Load in Delta table

The path to our existing Delta table is provided.

In [0]:
DeltaPath = userhome + "/delta/customer-data/"

Note that because we registered a global table associated with this Delta table, you should already be able to query this data with SQL. To see all your currently registered tables, click the `Data` icon on the left navigation bar.

<img src=https://s3-us-west-2.amazonaws.com/files.training.databricks.com/images/adbcore/data-button.png width=100px>

In [0]:
%sql

SELECT COUNT(*) FROM customer_data_delta LIMIT 5

count(1)
65545


Because we stored our data in Delta, our schema and partions are preserved. All we'll need to do is specify the format and the path.

In [0]:
# TODO

deltaDF = (spark.read
  .format("delta")
  .load(DeltaPath))

### Generate our BI table
We'll start out by just looking at our aggregate counts. Here, we group by both `"CustomerID"` and `"Country"`, as it is the combination of these two fields that is of interest to our BI team.

In [0]:
customerCounts = (deltaDF.groupBy("CustomerID", "Country")
  .count()
  .withColumnRenamed("count", "total_orders"))

display(customerCounts)

CustomerID,Country,total_orders
14443,United Kingdom,23
16858,United Kingdom,13
16143,United Kingdom,28
14299,United Kingdom,24
13370,United Kingdom,44
17547,United Kingdom,2
13047,United Kingdom,18
18256,United Kingdom,4
13599,United Kingdom,30
13496,United Kingdom,27


Clicking on the names of the various columns will allow us to quickly sort on different fields. You may notice that we have a large number of entries that are `null` for both `"CustomerID"` and `"Country"`. While in production, we would like to explore _why_ we are seeing these missing values, for now we'll just leave them as is and save out this DataFrame as a new Delta table.

Here, the path is provided for you.

In [0]:
CustomerCountsPath = userhome + "/delta/customer_counts/"

dbutils.fs.rm(CustomerCountsPath, True) #deletes Delta table if previously created

Out[25]: False

Here we'll write out our Delta table to the path provided above. Make sure the following settings are provided:
- `overwrite` (so that this code will work if you run it again)
- format as `delta`
- partition by `"Country"`
- save to `CustomerCountsPath`

In [0]:
# TODO

(customerCounts.write
  .mode("overwrite")
  .format("delta")
  .partitionBy("Country")
  .save(CustomerCountsPath))

We'll also register this Delta table as a Spark SQL table.

In [0]:
spark.sql("""
  DROP TABLE IF EXISTS customer_counts
""")

spark.sql("""
  CREATE TABLE customer_counts
  USING DELTA
  LOCATION '{}'
""".format(CustomerCountsPath))

Out[27]: DataFrame[]

Now our BI team can quickly query those data points they've expressed interest in.

In [0]:
%sql

SELECT *
FROM customer_counts limit 5

CustomerID,Country,total_orders
12415,Australia,47
16321,Australia,4
12393,Australia,22
12434,Australia,3
12388,Australia,19


Looking at your existing Delta table, you know that a large number of recent orders haven't been loaded in yet.

###  READ updated CSV data

Read the data into a DataFrame. We'll use the same schema that we used when creating our Delta table, which is supplied for you.

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

inputSchema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", StringType(), True),
  StructField("UnitPrice", DoubleType(), True),
  StructField("CustomerID", IntegerType(), True),
  StructField("Country", StringType(), True)
])

Read data in `newDataPath`. Re-use `inputSchema` as defined above. We'll name our DataFrame `newDataDF`.

In [0]:
# TODO

newDataPath = "/mnt/training/online_retail/outdoor-products/outdoor-products-small.csv"
newDataDF = (spark
  .read
  .option("header", "true")
  .schema(inputSchema)
  .csv(newDataPath)
)

Let's do the same aggregate count as above, on `"Country"` and `"CustomerID"`.

In [0]:
# TODO

newCustomerCounts = (newDataDF.groupBy("CustomerID", "Country").count()
  .withColumnRenamed("count", "total_orders"))

In [0]:
display(newCustomerCounts.limit(5))

CustomerID,Country,total_orders
21555,United States,2
20113,United States,4
20340,United States,5
21626,United States,1
21511,United States,1


### UPSERT new customer counts

Now that we've successfully loaded and aggregated our new data, we can upsert it in our existing Delta Lake.

First, we'll register it as a temp view.

In [0]:
newCustomerCounts.createOrReplaceTempView("new_customer_counts")

And now we can merge these new counts into our existing data.

In [0]:
%sql

MERGE INTO customer_counts
USING new_customer_counts
ON customer_counts.Country = new_customer_counts.Country
AND customer_counts.CustomerID = new_customer_counts.CustomerID
WHEN MATCHED THEN
  UPDATE SET total_orders = customer_counts.total_orders + new_customer_counts.total_orders
WHEN NOT MATCHED THEN
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
38138,14,0,38124


We can write a simple SQL query to confirm that this has worked.

In [0]:
%sql

SELECT SUM(total_orders) FROM customer_counts limit 5

sum(total_orders)
165544


### Update full records using append

We want to retain the full records being generated from our batch report in our existing non-aggregated Delta table.

In this case, we're assuming that the records we process at the end of each day are correct, and that batch processing will result in correct, stable records. We can safely write our table to the same file path using the append mode to insert these records.

**Note**: If our reports included changes to line items from previous days, we would want to write an UPSERT which would allow us to simultaneously update our changed records and insert new data.

In [0]:
# TODO

(newDataDF.write
  .format("delta")
  .mode("append")
  .save(DeltaPath))

Querying our table again shows that we've immediately updated.

In [0]:
%sql
SELECT COUNT(*) FROM customer_data_delta limit 5

count(1)
165544


## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/delta-batch.html#" target="_blank">Table Batch Read and Writes</a>